# ECON 490: Instrumental Variable Analysis (16)

## Prerequisites

1. Run OLS regressions.

## Learning Outcomes

1. Understand what an instrumental variable is and the conditions it must satisfy to address the endogeneity problem.
2. Implement a Two Stage Least Squares (2SLS) regression-based approach using an instrument. 
3. Describe the weak instrument problem.
4. Interpret the first stage test of whether or not the instrument is weak.

## 16.1 The linear IV model

Consider a case where we want to know the effect of education on earnings. We may want to estimate a model like the following

$$
Y_{i} = \alpha + \beta X_i + \epsilon_i
$$
where $Y_i$ is earnings of individual $i$ and $X_i$ is years of education of individual $i$.

A possible issue comes from omitted variable bias: it is possible that the decision to attend school is influenced by other individual characteristics that are also correlated with earnings. For example, think of individuals with high innate ability. They may want to enroll in school for longer and obtain higher-level degrees. Moreover, their employers may compensate them for their high ability, regardless of their years of schooling.

Instrumental variables can help us when there are hidden factors affecting both the treatment (in our case, years of education) and the outcome (in our case, earnings). The instrumental variables approach relies on finding something affecting the treatment and affecting the outcome solely through the treatment. In short, the instrument should satisfy two assumptions:
1. *relevance*: the instrument should be correlated with the explanatory variable; in our case, it should be correlated with the years of education $X_i$;
2. *exclusion restriction*: the instrument should be correlated with the dependent variable only through the explanatory variable; in our case, it should be correlated with $Y_i$ only through its correlation with $X_i$.

Let's say we have found an instrumental variable $Z_i$ for the variable $X_i$. Then, using an Instrumental Variable analyis implies estimating the following model
$$
\begin{align}
Y_i &= \alpha_1 + \beta X_i + u_i  \quad \text{(Structural Equation)}\\
X_i &= \alpha_2 + \gamma Z_i + e_i  \quad \text{(First Stage Equation)}
\end{align}
$$
where the two conditions we have seen above imply that:
1. $\gamma \neq 0$;
2. $Z_i$ is uncorrelated with $u_i$.

In practice, using an Instrumental Variable analysis often implies using a Two-Stages Least Square (2SLS) estimator. The two steps of the 2SLS are:
1. Estimate the First Stage Equation by OLS and obtain the predicted value of $X_i$. In this way, we have effectively split $X_i$ into
$$
X_i = \underbrace{\hat{X}_i}_\text{exogenous part} + \underbrace{\hat{e}_i}_\text{endogenous part} 
$$
where $\hat{X_i} \equiv \hat{\alpha_2} + \hat{\gamma} Z_i $.

2. Plug $\hat{X_i}$ instead of $X_i$ into the Structural Equation and estimate via OLS. We are then using the "exogenous" part of $X_i$ to capture $\beta$. 

<div class="alert alert-warning">

**Caution**: We can run 2SLS following the steps above, but when we want to do inference we need to be sure we're using the true residuals in the Structural equation $\hat{u}_i$. The built-in R command `ivreg` automatically give us the right residuals.
</div>

Let's see how to estimate this in R. Once again, we can use our fictional dataset simulating wages of workers in the years 1982-2012 in a fictional country.

In [ ]:
# Load in our packages
library(tidyverse)
library(haven)

In [ ]:
# Import dataset
fake_data <- read_dta("../econ490-stata/fake_data.dta")

In R, we can perform IV analysis with a 2SLS estimator by using the command `ivreg`. This command is a part of the package *AER* which we will have to install.  
The syntax is `ivreg(<Y> ~ <X> | <Z>, data=<data>)` where instead of `<Y>`, `<X>`, and `<Z>` you have to write the names of the corresponding Y, X, and Z variables of your model, and instead of `<data>` you write the name of your dataframe.

We now have to choose an instrumental variable that can work in our setting. A well-known example for an instrument for years of schooling is studied by Angrist and Krueger (1991): they propose that $Z$ is the quarter of birth. The premise behind their IV is that students are required to enter school in the *year they turn 6* but not necessarily when they are *already* 6 years old, creating a relationship between quarter of birth and schooling. At the same time, the time of the year one is born shouldn't  affect one's earnings aside from its effect on schooling.

Let's see how to estimate a simple IV in R using our data and `ivregr`.

In [ ]:
# Install AER package
install.packages("AER")
library(AER)

In [ ]:
modelIV <- ivreg(earnings ~ schooling | quarter_birth, data = fake_data)
summary(modelIV)

We obtain a standard output: value of the coefficients, standard errors, p-value, and 95% confidence intervals. From the regression output, years of schooling does not seem to have any effect on earnings. However, before trusting these results we should check that the two assumptions on IV are met in this case: relevance and exclusion restriction.

## 16.2 Weak instrument test
While we cannot really test for exclusion restriction, we can check whether our instrument is relevant. We do that by looking directly at the coefficients in the First Stage Equation. 

In R, we need to estimate the First Stage Equation and then perform a simple F-test.
Recall that our first stage is 
$$
X_i = \alpha_2 + \gamma Z_i + e_i
$$
where $X_i$ is years of education and $Z_i$ is quarter of birth.
We simply have to estimate this regression with a OLS. We have seen how to do it using the function `lm`.

In [ ]:
first_stage <- lm(schooling ~ quarter_birth, data = fake_data)
summary(first_stage)

Among the outputs given us by `summary`, we can see the F-statistic and its p-value. we can see that the instrumental variable we have chosen is not relevant for our explanatory variable $X$: *quarter_birth* is not correlated with *schooling*. Another indicator of lack of relevance is given by the F-statistic reported by R: as a rule of thumb, every time its value is less than 10, the instrument is not relevant. 

Whenever the correlation between $X$ and $Z$ is very close to zero (as in our case), we say we have a *weak instrument* problem. In practice, this problem will result in severe finite-sample bias and large variance in our estimates. Since our instrument is not valid, we cannot trust the results we have obtained so far.  

## 16.3 Wrap Up

In this module we studied the Linear IV model and how to estimate it using the Two-Stage Least Squares Method using `ivreg`. We learned that we can overcome the endogeneity problem when we have access to a different type of variable: instrumental variables. A good instrument must satisfy two important conditions:

1. It must be uncorrelated with the error term (also referred to as the exclusion restriction principle).
2. It must be correlated, after controlling for observables, with the variable of interest (there must be a first stage).

While condition 2 can be checked using regression results on the first stage, condition 1 is inherently not capable of being tested. Therefore, any project that uses instrumental variables must include a discussion, using contextual knowledge, of why condition 1 may hold. 

Finally, do not forget that for every endogenous variable in our regression, we require at least one instrument. For example, if we have a regression with 2 endogenous variables, we require at least 2 instrumental variables.

The new commands we have encountered in this module are summarized below:
|Command|Function|
|---------|--------|
|`ivreg`|It performs Instrumental Variable analysis using a Two-Stage Least Squares estimator by default.|